# Ray Tune

In [1]:
!pip install -U "ray[tune]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 MB 26.6 MB/s eta 0:00:00


# Env

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
!unzip -qq "/content/drive/MyDrive/MAGISTERKA/datasets/fog-combined.zip" -d "/content/datasets/"

In [4]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Setup

In [5]:
from pathlib import Path
from time import time

import torch
from torchvision.models import get_weight
from torchvision.transforms import v2

import numpy as np
import pandas as pd
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin
from typing import Any, TypeVar
from collections import namedtuple
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

import ray
from ray import tune


In [6]:
base_transform = v2.Compose([
    v2.Resize((256, 256), v2.InterpolationMode.BILINEAR),
    v2.CenterCrop((224, 224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])
transforms = {
    "grayscale": v2.Compose([
        base_transform,
        v2.Grayscale(num_output_channels=3)
    ]),
    "color": v2.Compose([
        base_transform,
    ]),
}

In [7]:
BASE_PATH = Path("/content")
DRIVE_PATH = Path("/content/drive/MyDrive/MAGISTERKA")

In [8]:
import sys
sys.path.insert(0, str(DRIVE_PATH / 'src'))
from cnn_model import CNNClassifier, get_dataloader

In [9]:
DATASET_PATHS = {
    'fog-detection': BASE_PATH / 'datasets/fog-detection-dataset-prepared',
    'fog-or-smog': BASE_PATH / 'datasets/fog-or-smog-detection-dataset-prepared',
    'foggy-cityscapes': BASE_PATH / 'datasets/foggy-cityscapes-image-dataset-prepared',
    'combined': BASE_PATH / 'datasets/fog-combined',
}

DATASET_NORMALIZATION = {
    'fog-detection': {'mean': [0.4850, 0.5044, 0.4878], 'std': [0.2631, 0.2524, 0.2793]},
    'fog-or-smog': {'mean': [0.5411, 0.5339, 0.5088], 'std': [0.2353, 0.2157, 0.2289]},
    'foggy-cityscapes': {'mean': [0.4169, 0.4507, 0.4173], 'std': [0.1860, 0.1835, 0.1820]},
    'combined': {'mean': [0.5017, 0.5087, 0.4826], 'std': [0.2259, 0.2118, 0.2227]}
}

In [10]:
def dataloader_to_df(dataloader):
  X = []
  y = []
  for images, labels in dataloader:
    for image, label in zip(images, labels):
      X.append(image.numpy().reshape(-1))
      y.append(label.numpy().item())
  df = pd.DataFrame({'features': X, 'class': y})
  return df

In [11]:
set(DATASET_PATHS.keys()) - {'combined'}

{'fog-detection', 'fog-or-smog', 'foggy-cityscapes'}

In [12]:
dfs_color = {
    dataset: {
        'train': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'train', transforms["grayscale"])),
        'val': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'val', transforms["grayscale"])),
        'test': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'test', transforms["grayscale"]))
    } for dataset in (set(DATASET_PATHS.keys()) - {'combined'})
}

# dfs_grayscale = {
#     dataset: {
#         'train': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'train', transforms["grayscale"])),
#         'val': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'val', transforms["grayscale"])),
#         'test': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'test', transforms["grayscale"]))
#     } for dataset in (set(DATASET_PATHS.keys()) - {'combined'})
# }

In [13]:
dfs_color['combined'] = {}
for split in ['train', 'val', 'test']:
  dfs_combined = [dfs_color[dataset][split] for dataset in (set(DATASET_PATHS.keys()) - {'combined'})]
  dfs_combined = pd.concat(dfs_combined)
  dfs_color['combined'][split] = dfs_combined

In [14]:
_T = TypeVar('_T', bound=ClassifierMixin)

def evaluate_classic_model(model: _T, X: Any, y: Any):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')
    confusion = confusion_matrix(y, y_pred)
    return namedtuple('Evaluation', ['accuracy', 'f1', 'precision', 'recall', 'confusion'])(accuracy, f1, precision, recall, confusion)

def _get_formatted_metric(values: list[float]):
  avg = sum(values) / len(values)
  std = (sum([(v - avg) ** 2 for v in values]) / len(values)) ** 0.5
  return f"{avg:.4f} ± {std:.4f}"

def train_model(
    model_cls: _T,
    model_kwargs: dict[str, Any],
    train_df: pd.DataFrame,
) -> _T:
    model = model_cls(**model_kwargs)
    model.fit(np.stack(train_df['features'].values), train_df['class'])
    return model

def run_model(
    model_cls: _T,
    model_kwargs: dict[str, Any],
    train_df: pd.DataFrame,
    # val_df: pd.DataFrame,
    test_dfs: dict[str, pd.DataFrame],
    repeat: int = 5,
    transform: str = "color",
    normalize: bool = False
):
  final_res = {
      dataset: {
          'accuracy': [],
          'f1': [],
          'precision': [],
          'recall': [],
      } for dataset in test_dfs.keys()
  }
  training_times = []
  for i in range(repeat):

    _start = time()
    model = train_model(
        model_cls=model_cls,
        model_kwargs=model_kwargs,
        train_df=train_df,
        # val_df=val_df
    )
    _end = time()
    training_times.append(_end - _start)
    for dataset, test_df in test_dfs.items():
      results = evaluate_classic_model(
          model=model,
          X=np.stack(test_df['features'].values),
          y=test_df['class']
      )
      final_res[dataset]['accuracy'].append(results.accuracy)
      final_res[dataset]['f1'].append(results.f1)
      final_res[dataset]['precision'].append(results.precision)
      final_res[dataset]['recall'].append(results.recall)
  print(final_res)
  latex_table = [
    [
      dataset,
      *[_get_formatted_metric(m) for m in metrics.values()]
    ] for dataset, metrics in final_res.items()
  ]
  latex_table_str = ""
  for line in latex_table:
    latex_table_str += " & ".join([str(l) for l in line]) + " \\\\\n"

  return latex_table_str


# Ray Tune

In [15]:
import ray
from ray import tune

In [16]:
def subsample_df(df: pd.DataFrame, frac: float, target_column: str='class') -> pd.DataFrame:
  dfs = []
  for class_name in df[target_column].unique():
    class_df = df[df[target_column] == class_name]
    class_df = class_df.sample(frac=frac)
    dfs.append(class_df)
  return pd.concat(dfs)

In [17]:
def train_model_tune(
    config,
    model_cls: _T,
    train_df: pd.DataFrame,
    val_df: pd.DataFrame
):
  model = model_cls(**config)

  model.fit(np.stack(train_df['features'].values), train_df['class'])
  val_metrics = evaluate_classic_model(model, np.stack(val_df['features'].values), val_df['class'])

  tune.report(
      {
    "accuracy": val_metrics.accuracy,
    "f1": val_metrics.f1,
    "precision": val_metrics.precision,
    "recall": val_metrics.recall
      }
  )

# SVC

In [18]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=LinearSVC,
        train_df=subsample_df(dfs_color['combined']['train'], frac=0.2),
        val_df=dfs_color['combined']['val']
    ),
    param_space={
        "C": tune.grid_search([0.01, 0.1, 0.4, 0.8, 1, 1.2, 2.0, 10]),
    }
)

results = tuner.fit()

2025-06-07 22:57:42,535	INFO worker.py:1888 -- Started a local Ray instance.
2025-06-07 22:57:45,073	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_22-57-39   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 8                                      |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_22-57-39
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_22-57-39_793194_4490/artifacts/2025-06-07_22-57-45/train_model_tune_2025-06-07_22-57-39/driver_artifacts`

Trial status: 8 PENDING
Current time: 2025-06-07 22:57:45. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+-------------------------------------------------+
|

(train_model_tune pid=7222) /usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
(train_model_tune pid=7222)   warnings.warn(



Trial train_model_tune_d3139_00005 completed after 1 iterations at 2025-06-07 23:06:06. Total running time: 8min 21s
+-------------------------------------------------------+
| Trial train_model_tune_d3139_00005 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              495.958 |
| time_total_s                                  495.958 |
| training_iteration                                  1 |
| accuracy                                      0.66921 |
| f1                                            0.64775 |
| precision                                     0.69764 |
| recall                                        0.65882 |
+-------------------------------------------------------+

Trial train_model_tune_d3139_00004 completed after 1 iterations at 2025-06-07 23:06:08. Total running time: 8min 22s
+-------------------------------------------------------+
| Trial tr

(train_model_tune pid=7224) /usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations. [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(train_model_tune pid=7224)   warnings.warn( [repeated 7x across cluster]
2025-06-07 23:06:12,486	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_22-57-39' in 0.0068s.



Trial train_model_tune_d3139_00000 completed after 1 iterations at 2025-06-07 23:06:12. Total running time: 8min 26s
+-------------------------------------------------------+
| Trial train_model_tune_d3139_00000 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              501.665 |
| time_total_s                                  501.665 |
| training_iteration                                  1 |
| accuracy                                      0.68448 |
| f1                                            0.66022 |
| precision                                     0.72615 |
| recall                                        0.67313 |
+-------------------------------------------------------+

Trial status: 8 TERMINATED
Current time: 2025-06-07 23:06:12. Total running time: 8min 26s
Logical resource usage: 1.0/8 CPUs, 0/0 GPUs
+------------------------------------------------

In [19]:
best_results = results.get_best_result(metric='f1', mode='max')
best_results.config

{'C': 0.01}

In [20]:
best_results = results.get_best_result(metric='f1', mode='max')

run_model(
    model_cls=LinearSVC,
    model_kwargs=best_results.config,
    train_df=dfs_color['combined']['train'],
    test_dfs={dataset: dfs_color[dataset]['test'] for dataset in dfs_color.keys()},
    repeat=5
)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'fog-detection': {'accuracy': [0.8113207547169812, 0.7924528301886793, 0.7830188679245284, 0.8113207547169812, 0.7924528301886793], 'f1': [0.7720430107526881, 0.744298245614035, 0.7299213470699014, 0.7720430107526881, 0.744298245614035], 'precision': [0.8823529411764706, 0.8735632183908046, 0.8693181818181819, 0.8823529411764706, 0.8735632183908046], 'recall': [0.7560975609756098, 0.7317073170731707, 0.7195121951219512, 0.7560975609756098, 0.7317073170731707]}, 'foggy-cityscapes': {'accuracy': [0.5858585858585859, 0.5858585858585859, 0.5909090909090909, 0.5909090909090909, 0.5858585858585859], 'f1': [0.5716999050332383, 0.570021186440678, 0.5761067794370291, 0.5806291347436132, 0.5716999050332383], 'precision': [0.5989417989417989, 0.60069402895776, 0.605668880455408, 0.6007918552036199, 0.5989417989417989], 'recall': [0.5858585858585859, 0.5858585858585859, 0.5909090909090909, 0.5909090909090909, 0.5858585858585859]}, 'fog-or-smog': {'accuracy': [0.8103448275862069, 0.810344827586206

'fog-detection & 0.7981 ± 0.0113 & 0.7525 ± 0.0168 & 0.8762 ± 0.0052 & 0.7390 ± 0.0146 \\\\\nfoggy-cityscapes & 0.5879 ± 0.0025 & 0.5740 ± 0.0039 & 0.6010 ± 0.0025 & 0.5879 ± 0.0025 \\\\\nfog-or-smog & 0.8103 ± 0.0014 & 0.8050 ± 0.0015 & 0.8347 ± 0.0016 & 0.8056 ± 0.0014 \\\\\ncombined & 0.7513 ± 0.0018 & 0.7403 ± 0.0026 & 0.7778 ± 0.0008 & 0.7426 ± 0.0022 \\\\\n'

# RFC

In [21]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=RandomForestClassifier,
        # train_df=dfs_color['combined']['train'],
        train_df=subsample_df(dfs_color['combined']['train'], frac=0.2),
        val_df=dfs_color['combined']['val']
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 100, 200]),
        "max_depth": tune.grid_search([1, 5, 10, 20, 50, None]),
    }
)

results = tuner.fit()

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-08_00-59-27   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 24                                     |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-08_00-59-27
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_22-57-39_793194_4490/artifacts/2025-06-08_00-59-27/train_model_tune_2025-06-08_00-59-27/driver_artifacts`

Trial status: 24 PENDING
Current time: 2025-06-08 00:59:28. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+---------------------------------------------------

2025-06-08 01:00:46,859	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-08_00-59-27' in 0.0120s.



Trial train_model_tune_d3e99_00023 completed after 1 iterations at 2025-06-08 01:00:46. Total running time: 1min 18s
+-------------------------------------------------------+
| Trial train_model_tune_d3e99_00023 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              34.4792 |
| time_total_s                                  34.4792 |
| training_iteration                                  1 |
| accuracy                                      0.82952 |
| f1                                            0.82936 |
| precision                                     0.82926 |
| recall                                        0.82998 |
+-------------------------------------------------------+

Trial status: 24 TERMINATED
Current time: 2025-06-08 01:00:46. Total running time: 1min 18s
Logical resource usage: 1.0/8 CPUs, 0/0 GPUs
+-----------------------------------------------

In [22]:
best_results = results.get_best_result(metric='f1', mode='max')

run_model(
    model_cls=RandomForestClassifier,
    model_kwargs=best_results.config,
    train_df=dfs_color['combined']['train'],
    test_dfs={dataset: dfs_color[dataset]['test'] for dataset in dfs_color.keys()},
    repeat=5
)

{'fog-detection': {'accuracy': [0.9811320754716981, 0.9811320754716981, 0.9811320754716981, 0.9811320754716981, 0.9811320754716981], 'f1': [0.9799242424242425, 0.9799242424242425, 0.9799242424242425, 0.9799242424242425, 0.9799242424242425], 'precision': [0.9850746268656716, 0.9850746268656716, 0.9850746268656716, 0.9850746268656716, 0.9850746268656716], 'recall': [0.975609756097561, 0.975609756097561, 0.975609756097561, 0.975609756097561, 0.975609756097561]}, 'foggy-cityscapes': {'accuracy': [0.7777777777777778, 0.7929292929292929, 0.7828282828282829, 0.7929292929292929, 0.8181818181818182], 'f1': [0.7732431025507549, 0.7900969618616678, 0.7780674087010921, 0.7900969618616678, 0.8154706430568499], 'precision': [0.801929688366419, 0.8096419327006039, 0.809375, 0.8096419327006039, 0.838048780487805], 'recall': [0.7777777777777778, 0.7929292929292929, 0.7828282828282829, 0.7929292929292929, 0.8181818181818182]}, 'fog-or-smog': {'accuracy': [0.9245689655172413, 0.9245689655172413, 0.920258

'fog-detection & 0.9811 ± 0.0000 & 0.9799 ± 0.0000 & 0.9851 ± 0.0000 & 0.9756 ± 0.0000 \\\\\nfoggy-cityscapes & 0.7929 ± 0.0139 & 0.7894 ± 0.0146 & 0.8137 ± 0.0125 & 0.7929 ± 0.0139 \\\\\nfog-or-smog & 0.9228 ± 0.0016 & 0.9228 ± 0.0016 & 0.9230 ± 0.0015 & 0.9235 ± 0.0016 \\\\\ncombined & 0.8974 ± 0.0036 & 0.8974 ± 0.0036 & 0.8988 ± 0.0034 & 0.8994 ± 0.0035 \\\\\n'

# XGBoost

In [23]:
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(
            train_model_tune,
            model_cls=XGBClassifier,
            train_df=subsample_df(dfs_color['combined']['train'], frac=0.2),
            val_df=dfs_color['combined']['val']
        ),
        resources={"cpu": 8}
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 100, 200]),
        "max_depth": tune.grid_search([1, 5, 10, 20, 50, None]),

    }
)

results = tuner.fit()

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-08_01-23-40   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 24                                     |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-08_01-23-40
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_22-57-39_793194_4490/artifacts/2025-06-08_01-23-40/train_model_tune_2025-06-08_01-23-40/driver_artifacts`

Trial status: 24 PENDING
Current time: 2025-06-08 01:23:40. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+---------------------------------------------------

RuntimeError: Caught unexpected exception: Task was killed due to the node running low on memory.
Memory on the node (IP: 172.28.0.12, ID: c354743bf54d08c6312c1688b1d5b0b7057cd5bcc127a754d2b04015) where the task (actor ID: bf248437ae1d3f597f20e15901000000, name=ImplicitFunc.__init__, pid=55325, memory used=32.49GB) was running was 48.51GB / 50.99GB (0.951332), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: e96f27866f4ed7365172dd790368947ee00f09031626ab1a330cd786) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.28.0.12`. To see the logs of the worker, use `ray logs worker-e96f27866f4ed7365172dd790368947ee00f09031626ab1a330cd786*out -ip 172.28.0.12. Top 10 memory users:
PID	MEM(GB)	COMMAND
55325	32.49	ray::ImplicitFunc.train
4490	4.33	/usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-792d03f0-cdd5...
103	0.10	/usr/bin/python3 /usr/

In [18]:
# best_results = results.get_best_result(metric='f1', mode='max')

run_model(
    model_cls=XGBClassifier,
    # model_kwargs=best_results.config,
    model_kwargs={
        "n_estimators": 50,
        "max_depth": 5
    },
    train_df=dfs_color['combined']['train'],
    test_dfs={dataset: dfs_color[dataset]['test'] for dataset in dfs_color.keys()},
    repeat=5
)

{'fog-or-smog': {'accuracy': [0.9137931034482759, 0.9137931034482759, 0.9137931034482759, 0.9137931034482759, 0.9137931034482759], 'f1': [0.9136631747390358, 0.9136631747390358, 0.9136631747390358, 0.9136631747390358, 0.9136631747390358], 'precision': [0.9138187774551411, 0.9138187774551411, 0.9138187774551411, 0.9138187774551411, 0.9138187774551411], 'recall': [0.9135416666666667, 0.9135416666666667, 0.9135416666666667, 0.9135416666666667, 0.9135416666666667]}, 'fog-detection': {'accuracy': [0.9716981132075472, 0.9716981132075472, 0.9716981132075472, 0.9716981132075472, 0.9716981132075472], 'f1': [0.9697344627391262, 0.9697344627391262, 0.9697344627391262, 0.9697344627391262, 0.9697344627391262], 'precision': [0.9779411764705883, 0.9779411764705883, 0.9779411764705883, 0.9779411764705883, 0.9779411764705883], 'recall': [0.9634146341463414, 0.9634146341463414, 0.9634146341463414, 0.9634146341463414, 0.9634146341463414]}, 'foggy-cityscapes': {'accuracy': [0.7828282828282829, 0.782828282

'fog-or-smog & 0.9138 ± 0.0000 & 0.9137 ± 0.0000 & 0.9138 ± 0.0000 & 0.9135 ± 0.0000 \\\\\nfog-detection & 0.9717 ± 0.0000 & 0.9697 ± 0.0000 & 0.9779 ± 0.0000 & 0.9634 ± 0.0000 \\\\\nfoggy-cityscapes & 0.7828 ± 0.0000 & 0.7808 ± 0.0000 & 0.7936 ± 0.0000 & 0.7828 ± 0.0000 \\\\\ncombined & 0.8880 ± 0.0000 & 0.8879 ± 0.0000 & 0.8877 ± 0.0000 & 0.8887 ± 0.0000 \\\\\n'

## Not Deep models

In [ ]:
run_model(
    model_cls=LinearSVC,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=1
)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9981
		F1: 0.9981
		Precision: 0.9982
		Recall: 0.9981
	Validation metrics:
		Accuracy: 0.7888
		F1: 0.7797
		Precision: 0.8215
		Recall: 0.7808


'combined & 0.8060 ± 0.0000 & 0.7988 ± 0.0000 & 0.8328 ± 0.0000 & 0.7983 ± 0.0000 \\\\\n'

In [ ]:
run_model(
    model_cls=LinearSVC,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=5
)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9989
		F1: 0.9989
		Precision: 0.9989
		Recall: 0.9989
	Validation metrics:
		Accuracy: 0.7939
		F1: 0.7859
		Precision: 0.8227
		Recall: 0.7864


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9993
		F1: 0.9993
		Precision: 0.9993
		Recall: 0.9992
	Validation metrics:
		Accuracy: 0.7888
		F1: 0.7801
		Precision: 0.8192
		Recall: 0.7810


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9981
		F1: 0.9981
		Precision: 0.9981
		Recall: 0.9982
	Validation metrics:
		Accuracy: 0.7964
		F1: 0.7907
		Precision: 0.8147
		Recall: 0.7903


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9981
		F1: 0.9981
		Precision: 0.9982
		Recall: 0.9981
	Validation metrics:
		Accuracy: 0.7913
		F1: 0.7843
		Precision: 0.8146
		Recall: 0.7844


'combined & 0.8128 ± 0.0036 & 0.8071 ± 0.0042 & 0.8333 ± 0.0018 & 0.8060 ± 0.0040 \\\\\n'

In [ ]:
run_model(
    model_cls=RandomForestClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=1
)

Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9288
		F1: 0.9287
		Precision: 0.9286
		Recall: 0.9296


'combined & 0.9128 ± 0.0000 & 0.9127 ± 0.0000 & 0.9136 ± 0.0000 & 0.9145 ± 0.0000 \\\\\n'

In [ ]:
run_model(
    model_cls=RandomForestClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=5
)

Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9237
		F1: 0.9236
		Precision: 0.9235
		Recall: 0.9245
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9186
		F1: 0.9185
		Precision: 0.9184
		Recall: 0.9194
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9186
		F1: 0.9185
		Precision: 0.9182
		Recall: 0.9189
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9237
		F1: 0.9236
		Precision: 0.9234
		Recall: 0.9242
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9211
		F1: 0.9210
		Preci

'combined & 0.9193 ± 0.0044 & 0.9193 ± 0.0044 & 0.9201 ± 0.0042 & 0.9210 ± 0.0042 \\\\\n'

In [ ]:
run_model(
    model_cls=XGBClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=1
)

Model - XGBClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9389
		F1: 0.9388
		Precision: 0.9386
		Recall: 0.9390


'combined & 0.9284 ± 0.0000 & 0.9282 ± 0.0000 & 0.9281 ± 0.0000 & 0.9283 ± 0.0000 \\\\\n'

In [ ]:
run_model(
    model_cls=XGBClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=5
)

Model - XGBClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9389
		F1: 0.9388
		Precision: 0.9386
		Recall: 0.9390
Model - XGBClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9389
		F1: 0.9388
		Precision: 0.9386
		Recall: 0.9390
